In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

import sys
sys.path.append('../../00.module')
import UA
import Proxy

# = = = = = = = = = = = = = = =

import pandas as pd

output_correct = pd.DataFrame()

output_error = pd.DataFrame()

input_ = pd.read_excel('./part_number.xlsx',
                       header=0,
                       dtype=str).fillna('')

input_ = input_[['Info']]
input_.drop_duplicates(ignore_index=True)

length = len(input_)

print('总数量：' + str(length))
print()

# = = = = = = = = = = = = = = =

import requests

from bs4 import BeautifulSoup

from lxml import etree

# = = = = = = = = = = = = = = =

import time
from datetime import datetime

# = = = = = = = = = = = = = = =

work = Queue()
for i in range(length):
    work.put_nowait(i)

def crawler():
    global output_correct, output_error
    
    while not work.empty():
        i = work.get_nowait()
        
        # = = = = = = = = = = = = = = =
        
        crawler_status = 'error'
        for _ in range(101):
            try:
                resp = requests.get(input_.loc[i, 'Info'],
                                    headers=UA.get_User_Agent_Requests(),
                                    proxies=Proxy.get_Proxy_Requests()).text
                
                # = = = = = = = = = = = = = = =
                
                soup = BeautifulSoup(resp, 'lxml')
                
                # = = = = = = = = = = = = = = =
                
                html = etree.HTML(str(soup))
                
                security_code = html.xpath('//div[@class="center"]/div')
                
                # = = = = = = = = = = = = = = =
                
                if len(security_code) != 0 and 'security code' in security_code[0].text:
                    continue
                
                # = = = = = = = = = = = = = = =

                list_tr = html.xpath('//table[@class="moreinfotable"]/tr')[1:]
                
                # = = = = = = = = = = = = = = =
                
                df_temp = pd.DataFrame([{'Info': input_.loc[i, 'Info']}])

                for tr in list_tr:
                    df_temp.loc[0, tr.xpath('./td')[0].text.strip()] = tr.xpath('./td')[1].text.strip()

                output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')

                # = = = = = = = = = = = = = = =
                
                crawler_status = 'ok'
                
                break
                
            except:
                time.sleep(0.3)
                continue

        if crawler_status != 'ok':
            df_temp = pd.DataFrame([{'Info': input_.loc[i, 'Info']}])
    
            output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')

        print(input_.loc[i, 'Info'] +  '  <->  [' + crawler_status + '] - 剩余数量：' + str(work.qsize()) + ' - 当前时间：' + datetime.now().strftime('%H:%M:%S'))

# = = = = = = = = = = = = = = =

list_task = []
for _ in range(20):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

print()
print('爬虫输出中')
output_correct = output_correct.drop_duplicates(ignore_index=True)
output_correct = output_correct.sort_values(by=['Info'], ascending=[True])
output_correct.to_excel('./info-' + datetime.now().strftime('%Y%m%d_%H%M%S') + '.xlsx', index=False)
if len(output_error) != 0:
    output_error = output_error.drop_duplicates(ignore_index=True)
    output_error = output_error.sort_values(by=['Info'], ascending=[True])
    output_error.to_excel('./info_error.xlsx', index=False)
    print()
    print('爬虫存在error')
print()
print('爬虫结束')

总数量：475

https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=8921204  <->  [ok] - 剩余数量：455 - 当前时间：15:44:12
https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=8304976  <->  [ok] - 剩余数量：454 - 当前时间：15:44:12
https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=8921200  <->  [ok] - 剩余数量：453 - 当前时间：15:44:12
https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=8304972  <->  [ok] - 剩余数量：452 - 当前时间：15:44:12
https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=10110000  <->  [ok] - 剩余数量：451 - 当前时间：15:44:12
https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=8304992  <->  [ok] - 剩余数量：450 - 当前时间：15:44:12
https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=8305000  <->  [ok] - 剩余数量：449 - 当前时间：15:44:12
https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=8304984  <->  [ok] - 剩余数量：448 - 当前时间：15:44:12
https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=8304980  <->  [ok] - 剩余数量：447 - 当前时间：15:44:13
https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=8305008  <->  [ok] - 剩余数量：446 - 当前时间：15:44:13
https://ww

KeyboardInterrupt
2024-02-19T07:44:14Z


KeyboardInterrupt: 